<a href="https://colab.research.google.com/github/Kennethkcpdhs/Honey_Pancake/blob/master/Python_with_SQL_Exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python with SQL Exercise 1

Create a SQLite3 database business.db with the following tables:
- Product(ProductID, ProductName, UnitPrice) 
- Customer(CustomerID, CustomerName, ContactNo)
- Order(OrderID, CustomerID, OrderDate)
- OrderDetails(OrderID, ProductID, Quantity)

In [ ]:
import csv
import sqlite3

conn = sqlite3.connect('customers.db')
conn.execute('''
CREATE TABLE IF NOT EXISTS "Product"(
    "ProductID" TEXT NOT NULL UNIQUE,
    "ProductName" TEXT NOT NULL,
    "UnitPrice" DECIMAL(5,2),
    PRIMARY KEY("ProductID")
);
''')
conn.commit()
conn.execute('''
CREATE TABLE IF NOT EXISTS "Customer"(
    "CustomerID" TEXT NOT NULL UNIQUE,
    "CustomerName" TEXT,
    "ContactNo" INTEGER,
    PRIMARY KEY("CustomerID")
);
''')
conn.commit()
conn.execute('''
CREATE TABLE IF NOT EXISTS "Order"(
    "OrderID" TEXT NOT NULL UNIQUE,
    "CustomerID" TEXT NOT NULL,
    "OrderDate" TEXT NOT NULL,
    PRIMARY KEY("OrderID"),
    FOREIGN KEY ("CustomerID") REFERENCES "Customer"("CustomerID")
);
''')
conn.commit()
conn.execute('''
CREATE TABLE IF NOT EXISTS "OrderDetails"(
    "OrderID" TEXT NOT NULL,
    "ProductID" TEXT,
    "Quantity" INTEGER,
    PRIMARY KEY("OrderID","ProductID"),
    FOREIGN KEY ("ProductID") REFERENCES "Product"("ProductID"),
    FOREIGN KEY ("OrderID") REFERENCES "Order"("OrderID")
);
''')
conn.commit()

Insert records into the respective tables using the following files:
- products.csv
- customers.csv
- orders.csv
- order_details.csv

In [ ]:

cursor = conn.cursor()
with open('customers.csv','r') as customers_file:
    csv_reader = csv.reader(customers_file,delimiter=",")
    customerdata = []

    for row in csv_reader:
        customerdata.append(row)
        
    for data in customerdata[1:]:
        cursor.execute("INSERT INTO Customer(CustomerID,CustomerName,ContactNo) VALUES (?,?,?)",(data[0],data[1],data[2]))
        conn.commit()

with open('products.csv','r') as products:
    csv_reader = csv.reader(products,delimiter=",")
    products = []

    for row in csv_reader:
        products.append(row)
    

    for product in products[1:]:
        cursor.execute("INSERT INTO Product(ProductID,ProductName,UnitPrice) VALUES (?,?,?)",(product[0],product[1],product[2]))
        conn.commit()


with open('orders.csv','r') as f:
    csv_reader = csv.reader(f,delimiter=",")
    orders = []

    for row in csv_reader:
        orders.append(row)
    
    for order in orders[1:]:
        cursor.execute("INSERT INTO 'Order'(OrderID,CustomerID,OrderDate) VALUES (?,?,?)",(order[0],order[1],order[2]))
        conn.commit()



with open('order_details.csv','r') as order_details:
    csv_reader = csv.reader(order_details,delimiter=",")
    orderdetails = []

    for row in csv_reader:
        orderdetails.append(row)
    
    for orderdetail in orderdetails[1:]:
        cursor.execute("INSERT INTO OrderDetails(OrderID,ProductID,Quantity) VALUES (?,?,?)",(orderdetail[0],orderdetail[1],orderdetail[2]))
        conn.commit()
cursor.close()

Write SQL query statements to determine:
- number of orders made on a particular day entered by a user
- total revenue on a particular day
- number of orders made by a particular customer
- product(s) with lowest sales
- best selling product(s) over a period of time (start to end dates inclusive)

In [ ]:
import csv
import sqlite3
conn = sqlite3.connect('customers.db')
cursor = conn.cursor()

In [ ]:
#Task 1
username = 'Karin Josephs'
day = '1996-07-05'
customerid = cursor.execute("SELECT CustomerID FROM Customer WHERE CustomerName=(?)",(username,)).fetchone()[0]
print(customerid)
#items = cursor.execute("SELECT OrderID FROM 'Order' WHERE CustomerID=?",(str(customerid),))
items = cursor.execute("SELECT COUNT(OrderID) FROM 'Order' WHERE OrderDate=? AND CustomerID=?",(str(day),str(customerid)))
items = items.fetchall()[0][0]
print("No.of orders made: ",items)

TOMSP
No.of orders made:  1


In [ ]:
#Task 2
newday = '1996-07-08'
bought = cursor.execute("SELECT OrderID FROM 'Order' WHERE OrderDate=?",(str(newday),)).fetchall()
print(bought)
cost = 0
for items in bought:
    items = items[0]
    print(items)
    quantity = cursor.execute("SELECT Quantity FROM OrderDetails WHERE OrderID=?",(items,)).fetchone()[0]
    productid = cursor.execute("SELECT ProductID FROM OrderDetails WHERE OrderID=?",(items,)).fetchone()[0]
    price = cursor.execute("SELECT UnitPrice FROM Product WHERE ProductID=?",(str(productid),)).fetchone()
    #print(quantity,productid,price)
    cost += (int(price[0])*int(quantity))
    
print(cost)

[('10250',), ('10251',)]
10250
10251
216


In [ ]:
#Task 3
username = 'Karin Josephs'
customerid = cursor.execute("SELECT CustomerID FROM Customer WHERE CustomerName=(?)",(username,)).fetchone()[0]
items = cursor.execute("SELECT Count(OrderID) FROM 'Order' WHERE CustomerID=?",(str(customerid),)).fetchall()[0][0]
print("Number of orders made: ",items)

Number of orders made:  6


In [ ]:
#Task 4
lowestquantity = """
SELECT ProductID, SUM(Quantity) FROM OrderDetails GROUP BY ProductID ORDER BY SUM(Quantity);
"""
minquantity = cursor.execute(lowestquantity).fetchall()
firstq = minquantity[0][1]
print(minquantity)
print("Products: ",end='')
for i in range(len(minquantity)):
    if minquantity[i][1] == firstq:
        print(minquantity[i][0],end=' ')
    else:
        break

[('9', 95), ('15', 122), ('37', 125), ('48', 138), ('67', 184), ('50', 235), ('66', 239), ('73', 293), ('32', 297), ('74', 297), ('5', 298), ('6', 301), ('20', 313), ('25', 318), ('3', 328), ('12', 344), ('22', 348), ('27', 365), ('8', 372), ('14', 404), ('57', 434), ('63', 445), ('4', 453), ('47', 485), ('52', 500), ('34', 506), ('45', 508), ('49', 520), ('58', 534), ('18', 539), ('46', 548), ('23', 580), ('43', 580), ('44', 601), ('61', 603), ('30', 612), ('38', 623), ('28', 640), ('42', 697), ('11', 706), ('69', 714), ('53', 722), ('19', 723), ('64', 740), ('10', 742), ('65', 745), ('29', 746), ('26', 753), ('33', 755), ('54', 755), ('7', 763), ('77', 791), ('39', 793), ('68', 799), ('36', 805), ('72', 806), ('70', 817), ('1', 828), ('35', 883), ('51', 886), ('13', 891), ('55', 903), ('17', 978), ('41', 981), ('76', 981), ('21', 1016), ('2', 1057), ('71', 1057), ('62', 1083), ('40', 1103), ('24', 1125), ('75', 1155), ('16', 1158), ('56', 1263), ('31', 1397), ('59', 1496), ('60', 157

In [ ]:
#Task 5
start = '1996-07-04'
end = '1996-07-08'

querystring = """
SELECT ProductID, SUM(Quantity) FROM OrderDetails WHERE OrderID IN (
SELECT OrderID From 'Order' WHERE OrderDate>=? AND Orderdate<=?
) GROUP BY ProductID ORDER BY SUM(Quantity) DESC;
"""
maxquantity = cursor.execute(querystring,(str(start),str(end))).fetchall()
print(maxquantity)                          
firstq = maxquantity[0][1]

print("Best Selling Products in period {0} to {1}: ".format(start,end),end='')
for i in range(len(maxquantity)):
    if maxquantity[i][1] == firstq:
        print(maxquantity[i][0],end=' ')
    else:
        break

[('51', 75), ('65', 35), ('57', 15), ('11', 12), ('41', 10), ('42', 10), ('14', 9), ('22', 6), ('72', 5)]
Best Selling Products in period 1996-07-04 to 1996-07-08: 51 